<a href="https://colab.research.google.com/github/leidysguerrero120/Trabajo-Anal-tica-3---Marketing/blob/main/Modelos_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas plotly


In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact
from sklearn import neighbors
import joblib
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_final')

cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

for table in tables:
    print(table[0])

ratings
movies
usuarios_sel
movies_sel
ratings_final
movies_final
full_ratings


***Sistemas basados en popularidad***

In [ ]:
import plotly.express as px
import pandas as pd

###Obtener las 10 películas mejor calificadas
query = """
SELECT movie_title,
       AVG(rating) AS avg_rat,
       COUNT(*) AS w_num
FROM full_ratings
WHERE rating <> 0
GROUP BY movie_title
ORDER BY avg_rat DESC
LIMIT 10
"""
top_movies = pd.read_sql(query, conn)

fig = px.bar(
    top_movies,
    x='avg_rat',
    y='movie_title',
    orientation='h',
    color='avg_rat',
    title='Top 10 películas mejor calificadas',
    labels={'avg_rat': 'Calificación Promedio', 'movie_title': 'Película'},
    color_continuous_scale=px.colors.sequential.Plasma
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
    ),
    yaxis=dict(
        categoryorder='total ascending',
    ),
    title_font=dict(size=24, family='Arial', color='darkblue'),
    font=dict(family='Arial', size=12, color='black'),
)

fig.show()


La película **The Shawshank Redemption (1994)** destaca con un promedio de calificación de 4.44 y 219 calificaciones. Su narrativa sobre la esperanza y la redención la ha consolidado como un clásico atemporal, resonando profundamente en la audiencia a lo largo del tiempo.

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_final')


query = """
SELECT movie_title,
       AVG(CASE WHEN rating = 0 THEN NULL ELSE rating END) AS avg_rat,
       COUNT(*) AS w_num
FROM full_ratings
GROUP BY movie_title
ORDER BY w_num DESC
LIMIT 10
"""


result = pd.read_sql(query, conn)


fig = px.bar(
    result,
    x='w_num',
    y='movie_title',
    title='Top 10 Películas Más Vistas',
    labels={'w_num': 'Número de Calificaciones', 'movie_title': 'Título de la Película'},
    text='avg_rat',
    color='avg_rat',
    color_continuous_scale=px.colors.sequential.Viridis
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    xaxis_title='Número de Calificaciones',
    yaxis_title='Título de la Película',
    template='plotly_white',
    title_x=0.5
)

fig.show()

conn.close()


**Forrest Gump (1994)** es la película con más calificaciones en la base de datos, lo que indica que es una de las más populares o vistas. Sin embargo, a pesar de su popularidad, su promedio de calificación es **4.12**, que no es lo suficientemente alto para aparecer entre las **mejor calificadas** en el top 10. Esto sugiere que, aunque muchas personas la han visto y calificado, las opiniones sobre ella varían más en comparación con las películas mejor calificadas que alcanzaron promedios más altos.

Esto quiere decir, que el gran número de calificaciones refleja popularidad, pero su promedio indica que no tiene la mejor valoración entre todos los usuarios. Esto destaca que una película puede ser muy vista, pero no necesariamente estar entre las mejor valoradas.

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_final')

query = """
SELECT year,
       movie_title,
       AVG(rating) AS avg_rating
FROM full_ratings
GROUP BY year, movie_title
ORDER BY year DESC, avg_rating DESC
"""

best_movies_by_year = pd.read_sql(query, conn)

print(best_movies_by_year)

conn.close()

     year                             movie_title  avg_rating
0    2016                         Deadpool (2016)    3.755556
1    2014          Guardians of the Galaxy (2014)    4.039216
2    2014                     Interstellar (2014)    3.885965
3    2014        Grand Budapest Hotel, The (2014)    3.775510
4    2013         Wolf of Wall Street, The (2013)    3.847826
..    ...                                     ...         ...
431  1941                     Citizen Kane (1941)    4.030303
432  1940                         Fantasia (1940)    3.770833
433  1940                        Pinocchio (1940)    3.410714
434  1939                Wizard of Oz, The (1939)    3.886364
435  1937  Snow White and the Seven Dwarfs (1937)    3.605634

[436 rows x 3 columns]


In [ ]:
best_movies_by_year['decade'] = (best_movies_by_year['year'] // 10) * 10

decade_avg = best_movies_by_year.groupby('decade')['avg_rating'].mean().reset_index()

fig = px.bar(
    decade_avg,
    x='decade',
    y='avg_rating',
    title='Promedio de Calificación de Películas por Década',
    labels={'avg_rating': 'Promedio de Calificación', 'decade': 'Década'},
    color='avg_rating',
    color_continuous_scale=px.colors.sequential.Plasma
)

fig.show()


Al analizar las calificaciones promedio de las películas a través de los años, se observa un contraste interesante entre **Deadpool** (2016) y **Snow White and the Seven Dwarfs** (1937). A pesar de ser la película más reciente, **Deadpool** solo alcanzó un promedio de calificación de **3.71**, lo que indica que, aunque popular, no fue tan bien valorada por los críticos o el público. Por otro lado, **Snow White and the Seven Dwarfs**, a pesar de su antigüedad y su estatus como clásico, presenta un promedio aún más bajo de **3.60**.

Esto plantea preguntas sobre la evolución de los gustos del público a lo largo del tiempo y cómo las expectativas cambian en relación a las producciones cinematográficas. Mientras que **Deadpool** logró una gran aceptación en taquilla, su calificación sugiere que su contenido podría no haber resonado con todos los críticos. En contraste, la película de 1937, considerada pionera en la animación, podría ser apreciada por su innovación más que por su calidad en comparación con estándares contemporáneos.

Este análisis resalta la complejidad de las calificaciones en la industria del cine, donde la popularidad no siempre se traduce en una alta valoración crítica.